# [LVV-T1994 - M2 Minimum Functionality Test]

This notebook executes the test case above.  
It is used only for data collection.   
Please, use the `./LVV-T1991-M2_min_function_test_EFD.ipynb` notebook to analyse the collected data.

[LVV-T1994 - M2 Minimum Functionality Test]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T1994

In [1]:
from lsst.ts import salobj
import asyncio
import os

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

In [2]:
summit = 1 #use this for summit testing
#summit = 0 #use this for NCSA

In [3]:
if summit:
    print(os.environ["OSPL_URI"])
    print(os.environ["LSST_DDS_PARTITION_PREFIX"])
    print(os.environ["LSST_DDS_DOMAIN_ID"])
else:
    import os
    print(os.environ["OSPL_URI"])
    if os.environ.get("LSST_DDS_ALIGNER", "false") != "false":
        print("LSST_DDS_ALIGNER is mis-configured")

file:///home/b1quint/WORK/ts_ddsconfig/config/ospl-shmem.xml
summit
0


In [4]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
#await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
m2 = salobj.Remote(script.domain, "MTM2")
print(f'time to start is {datetime.now() - start_time} [s]')

await m2.start_task

time to start is 0:00:00.145021 [s]


In [5]:
#As long as you get something for the payload its OK. we dont' care about h.heartbeat
await m2.evt_heartbeat.next(flush=True, timeout=5)

In [12]:
state = await m2.evt_summaryState.aget(timeout=5)
print('starting m2 state', state.summaryState,pd.to_datetime(state.private_sndStamp, unit='s'))
if not state.summaryState == 2:
    await salobj.set_summary_state(m2, salobj.State.ENABLED) #enable m2
    await asyncio.sleep(3)
    state = await m2.evt_summaryState.aget(timeout=5)
    print('m2 state', state.summaryState,pd.to_datetime(state.private_sndStamp, unit='s'))

starting m2 state 3 2022-05-11 15:49:34.655880704
m2 state 2 2022-05-11 15:50:34.486831360


In [13]:
fa = [0]*72
ft = [0]*6
await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)

In [14]:
holdTime = 5 #the M2 simulator publishes telemetry at 1Hz
fa = [0]*72
ft = [0]*6
start_time = datetime.now()
print('start time: ', start_time)
for i in range(72):
    fa[i] = 20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print('axial %d: +'%i, end='')
    await asyncio.sleep(holdTime)
    fa[i] = -20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' -', end='')
    await asyncio.sleep(holdTime)
    fa[i] = 0
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' 0 ', end='')
    await asyncio.sleep(holdTime)
    
for i in range(6):
    ft[i] = 20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print('tangent A%d: +'%i, end='')
    await asyncio.sleep(holdTime)
    ft[i] = -20
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' -', end='')
    await asyncio.sleep(holdTime)
    ft[i] = 0
    await m2.cmd_applyForces.set_start(axial=fa, tangent=ft)
    print(' 0 ', end='')
    await asyncio.sleep(holdTime)    
print(f'\n time elapsed: {datetime.now() - start_time} [s]')

start time:  2022-05-11 15:55:21.646214
axial 0: + - 0 axial 1: + - 0 axial 2: + - 0 axial 3: + - 0 axial 4: + - 0 axial 5: + - 0 axial 6: + - 0 axial 7: + - 0 axial 8: + - 0 axial 9: + - 0 axial 10: + - 0 axial 11: + - 0 axial 12: + - 0 axial 13: + - 0 axial 14: + - 0 axial 15: + - 0 axial 16: + - 0 axial 17: + - 0 axial 18: + - 0 axial 19: + - 0 axial 20: + - 0 axial 21: + - 0 axial 22: + - 0 axial 23: + - 0 axial 24: + - 0 axial 25: + - 0 axial 26: + - 0 axial 27: + - 0 axial 28: + - 0 axial 29: + - 0 axial 30: + - 0 axial 31: + - 0 axial 32: + - 0 axial 33: + - 0 axial 34: + - 0 axial 35: + - 0 axial 36: + - 0 axial 37: + - 0 axial 38: + - 0 axial 39: + - 0 axial 40: + - 0 axial 41: + - 0 axial 42: + - 0 axial 43: + - 0 axial 44: + - 0 axial 45: + - 0 axial 46: + - 0 axial 47: + - 0 axial 48: + - 0 axial 49: + - 0 axial 50: + - 0 axial 51: + - 0 axial 52: + - 0 axial 53: + - 0 axial 54: + - 0 axial 55: + - 0 axial 56: + - 0 axial 57: + - 0 axial 58: + - 0 axial 59: + - 0 axial 60: 

In [15]:
#if we started with disabled state, we need to put it back
await salobj.set_summary_state(m2, salobj.State.STANDBY) 

[<State.ENABLED: 2>, <State.DISABLED: 1>, <State.STANDBY: 5>]

In [17]:
print(f'time to end is {datetime.now()} [s]')

time to end is 2022-05-11 16:21:00.977628 [s]
